## PIP INSTALL, ERROR 발생에 필요한 코드

In [1]:
!pip install opencv-python

In [4]:
!apt-get -y install libgl1-mesa-glx




libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [5]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
!pip install efficientnet_pytorch

## 라이브러리 불러오기

In [1]:
import random
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet50
from efficientnet_pytorch import EfficientNet

from data import MaskDataset

## SEED 설정

In [4]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

NameError: name 'torch' is not defined

## 데이터 경로 설정

In [5]:
# train_root = '../input/data/processed0/train/images'
# valid_root = '../input/data/processed0/valid/images'
# train_root = '../input/data/processed1/train/images'
# valid_root = '../input/data/processed1/valid/images'
# train_root = '../input/data/processed2/train/images'
# valid_root = '../input/data/processed2/valid/images'
# train_root = '../input/data/processed3/train/images'
# valid_root = '../input/data/processed3/valid/images'
train_root = '../input/data/processed4/train/images'
valid_root = '../input/data/processed4/valid/images'
input_size = 224

## transform 적용해서 train, valid에 적용하기

In [6]:
# transform_train = A.Compose([
#     A.CenterCrop(384, 384, p=0.5),
#     A.Resize(224, 224, p=1.0),
#     A.CLAHE(p=1.0),
#     A.HorizontalFlip(p=0.5),
#     A.Normalize(mean=(0.5602, 0.5241, 0.5015), std=(0.2332, 0.2430, 0.2457), p=1.0),    
#     ToTensorV2(),   
# ])

transform_train = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop((384, 384))], p = 0.5),
    transforms.RandomApply([transforms.ColorJitter(0.1, 0.1)], p = 0.5),
    transforms.RandomApply([transforms.RandomRotation(10)], p = 0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5602, 0.5241, 0.5015), std=(0.2332, 0.2430, 0.2457))
])

NameError: name 'transforms' is not defined

In [6]:
# transform_valid = A.Compose([
# #     A.CenterCrop(384, 384, p=0.5),
#     A.Resize(224, 224, p=1.0),
#     A.CLAHE(p=1.0),
# #     A.HorizontalFlip(p=0.5),
#     A.Normalize(mean=(0.5602, 0.5241, 0.5015), std=(0.2332, 0.2430, 0.2457), p=1.0),    
#     ToTensorV2(),   
# ])

transform_valid = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5602, 0.5241, 0.5015), std=(0.2332, 0.2430, 0.2457))
])

In [7]:
train_dataset = MaskDataset(train_root, input_size, transform_train, phase='train')
valid_dataset = MaskDataset(valid_root, input_size, transform_valid, phase='valid')


#dataset = MaskDataset(train_root, input_size)
# n_val = int(len(dataset) * 0.2)
# n_train = len(dataset) - n_val
# print(n_val, n_train)
# train_dataset, val_dataset = random_split(dataset, [n_train, n_val])
# train_dataset.dataset.set_transform(transform_train)
# val_dataset.dataset.set_transform(transform_valid)

## 증강 후 데이터 양 확인

In [8]:
len(train_dataset)

NameError: name 'train_dataset' is not defined

In [9]:
len(valid_dataset)

3759

## 데이터셋 데이터로더로 불러오기 

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=torch.cuda.is_available(), num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, pin_memory=torch.cuda.is_available(), num_workers=8)

In [11]:
len(train_loader.sampler)

51272

## EfficientNet 모델 정의

In [9]:
pretrained = True

# model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=18)
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=18)
model.cuda()


NameError: name 'EfficientNet' is not defined

## Resnet50 정의

In [10]:
class Resnet50(nn.Module):
    def __init__(self, num_classes=18, pretrained=False):
        super(Resnet50, self).__init__()
        self.model = resnet50()
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, num_classes)
  
    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
model = Resnet50(num_classes=18, pretrained=True)
model.cuda()

Resnet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

## loss 정의

In [10]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

NameError: name 'nn' is not defined

In [ ]:
# criterion = FocalLoss()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, 2, gamma=0.5)

## 파라미터 정의

In [11]:
lr = 1e-4
input_size = 224
log_interval = 200
Epochs = 10
save_dir = '../model/efficientb0_processed4_train_epochs10_aug'
batch_size = 32

## training

In [12]:
# Main
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0
best_val_loss = np.inf
for epoch in range(Epochs):
    model.train()
    loss_value = 0
    matches = 0
    # Training
    for iter, (img, label) in enumerate(train_loader, 0):
    # optimizer에 저장된 미분값을 0으로 초기화

        # GPU 연산을 위해 이미지와 정답 tensor를 GPU로 보내기 (필요한 경우, 변수의 type도 수정해주세요)
        img, label = img.float().cuda(), label.long().cuda()
        optimizer.zero_grad()

        # 모델에 이미지 forward
        pred_logit = model(img)
        preds = torch.argmax(pred_logit, dim=-1)

        # loss 값 계산
        loss = criterion(pred_logit, label)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Accuracy 계산
        loss_value += loss.item()
        matches += (preds == label).sum().item()

        # Validation for every 20 epoch
        if (iter + 1) % log_interval == 0:
            train_loss = loss_value / log_interval
            train_acc = matches / batch_size / log_interval
            print(
                f'Epoch[{epoch}/{Epochs}]({iter + 1}/{len(train_loader)}) || '
                f'training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} ||'
            )

            loss_value = 0
            matches = 0
    
    scheduler.step()

    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for (img, label) in valid_loader:
            img, label = img.float().cuda(), label.long().cuda()

            pred_logit = model(img)
            preds = torch.argmax(pred_logit, dim=-1)

            loss_item = criterion(pred_logit, label).item()
            acc_item = (label == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_dataset)
        best_val_loss = min(best_val_loss, val_loss)

        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(model.state_dict(), f"{save_dir}/best.pth")
            best_val_acc = val_acc
        torch.save(model.state_dict(), f"{save_dir}/{epoch}.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        print()

NameError: name 'np' is not defined